In [1]:
from src.data import data_generation
import mindspore
from mindspore import nn, ops, Tensor
from mindspore.train import Model
import mindspore.numpy as mnp
import numpy as np

In [2]:
data = data_generation()

In [3]:
train_data, train_labels = data.train_data()
test_data , test_labels  = data.test_data()
print(train_data.shape)
print(train_labels.shape)

(18, 3)
(18, 3, 1)


In [4]:
x = train_data[:,0]
y = train_data[:,1]
z = train_data[:,2]
Bx = train_labels[:,0,0]
By = train_labels[:,1,0]
Bz = train_labels[:,2,0]

In [5]:
def sine_activation(x):
    return ops.Sin()(x)

class PINN(nn.Cell):
    def __init__(self, units, activation=sine_activation):
        super(PINN, self).__init__()
        self.hidden_layer1 = nn.Dense(3, units)
        self.hidden_layer2 = nn.Dense(units, units)
        self.hidden_layer3 = nn.Dense(units, units)
        self.hidden_layer4 = nn.Dense(units, units)
        self.hidden_layer5 = nn.Dense(units, 3)
        self.activation = activation

    def construct(self, x, y, z):
        inputs = ops.cat((x.reshape(-1,1), y.reshape(-1,1), z.reshape(-1,1)), axis=1)
        h1 = self.hidden_layer1(inputs)
        h1 = self.activation(h1)
        h2 = self.hidden_layer2(h1)
        h2 = self.activation(h2)
        h3 = self.hidden_layer3(h2 + h1)
        h3 = self.activation(h3)
        h4 = self.hidden_layer4(h3 + h2 + h1)
        h4 = self.activation(h4)
        h5 = self.hidden_layer5(h4 + h3 + h2 + h1)
        return h5[:,0], h5[:,1], h5[:,2]

In [6]:
network = PINN(32)
Bx_pred, By_pred, Bz_pred = network(x,y,z)

In [7]:
print(Bx_pred.shape)
print(Bx.shape)

(18,)
(18,)


In [8]:
optimizer = nn.optim.Adam(network.trainable_params(), learning_rate=0.001)

In [9]:
def compute_curl(networks, x, y, z, epsilon=1e-5):  
    # 计算向量场在 (x, y, z) 点的值  
    Fx0, Fy0, Fz0 = networks(x, y, z)  
      
    # 计算向量场在 (x+epsilon, y, z) 点的值，用于计算 partial_Fx_partial_x  
    Fx_eps_x, Fy_eps_x, Fz_eps_x = networks(x + epsilon, y, z)  
      
    # 计算所有需要的偏导数（使用数值微分）  
    partial_Fx_partial_x = (Fx_eps_x - Fx0) / epsilon  
    partial_Fy_partial_x = (Fy_eps_x - Fy0) / epsilon  
    partial_Fz_partial_x = (Fz_eps_x - Fz0) / epsilon 
    
    Fx_eps_y, Fy_eps_y, Fz_eps_y = networks(x, y + epsilon, z)  
    partial_Fx_partial_y = (Fx_eps_y - Fx0) / epsilon  
    partial_Fy_partial_y = (Fy_eps_y - Fy0) / epsilon  
    partial_Fz_partial_y = (Fz_eps_y - Fz0) / epsilon  
      
    Fx_eps_z, Fy_eps_z, Fz_eps_z = networks(x, y, z + epsilon)  
    partial_Fx_partial_z = (Fx_eps_z - Fx0) / epsilon  
    partial_Fy_partial_z = (Fy_eps_z - Fy0) / epsilon  
    partial_Fz_partial_z = (Fz_eps_z - Fz0) / epsilon  
      
    # 计算旋度  
    curl_x = partial_Fz_partial_y - partial_Fy_partial_z  
    curl_y = partial_Fx_partial_z - partial_Fz_partial_x  
    curl_z = partial_Fy_partial_x - partial_Fx_partial_y  
      
    return ops.Sqrt()(ops.ReduceMean()(curl_x**2+curl_y**2+curl_z**2))

def compute_div(networks, x, y, z, epsilon=1e-5):  
    # 计算向量场在 (x, y, z) 点的值  
    Fx0, Fy0, Fz0 = networks(x, y, z)  
      
    # 计算向量场在 (x+epsilon, y, z) 点的值，用于计算 partial_Fx_partial_x  
    Fx_eps_x, Fy_eps_x, Fz_eps_x = networks(x + epsilon, y, z)  
    partial_Fx_partial_x = (Fx_eps_x - Fx0) / epsilon 
    
    Fx_eps_y, Fy_eps_y, Fz_eps_y = networks(x, y + epsilon, z)  
    partial_Fy_partial_y = (Fy_eps_y - Fy0) / epsilon  
      
    Fx_eps_z, Fy_eps_z, Fz_eps_z = networks(x, y, z + epsilon)  
    partial_Fz_partial_z = (Fz_eps_z - Fz0) / epsilon   
      
    return ops.Sqrt()(ops.ReduceMean()(partial_Fx_partial_x**2 + partial_Fy_partial_y**2 + partial_Fy_partial_y**2))
    
class PINN_Loss(nn.Cell):
    def __init__(self, N_f, L, addBC, epsilon):
        super(PINN_Loss, self).__init__()
        self.N_f = N_f
        self.L = L
        self.addBC = addBC
        self.epsilon = epsilon

    def construct(self, x, y, z, Bx, By, Bz, model):
        L = self.L
        N_f = self.N_f
        loss_BC_div = compute_div(model, x, y, z, self.epsilon)
        loss_BC_cul = compute_curl(model, x, y, z, self.epsilon)
        pred_x, pred_y, pred_z = model(x,y,z)
        
        x_f = Tensor(np.random.uniform(-L/2, L/2, (N_f, 1))).astype(mindspore.float32)
        y_f = Tensor(np.random.uniform(-L/2, L/2, (N_f, 1))).astype(mindspore.float32)
        z_f = Tensor(np.random.uniform(-L/2, L/2, (N_f, 1))).astype(mindspore.float32)

        loss_div = compute_div(model, x_f, y_f, z_f)
        loss_cul = compute_curl(model, x_f, y_f, z_f)

        loss_b_x = ops.ReduceMean()(ops.Pow()(pred_x - Bx, 2))
        loss_b_y = ops.ReduceMean()(ops.Pow()(pred_y - By, 2))
        loss_b_z = ops.ReduceMean()(ops.Pow()(pred_z - Bz, 2))
        loss_b = ops.Sqrt()(loss_b_x + loss_b_y + loss_b_z)

        if self.addBC:
            loss = loss_b + loss_div + loss_cul + loss_BC_div + loss_BC_cul
        else:
            loss = loss_b + loss_div + loss_cul
        return loss_b, loss_div, loss_cul, loss_BC_div, loss_BC_cul, loss



In [10]:
loss_fn = PINN_Loss(256, 0.5, 1, 1e-5)
loss_fn(x,y,z, Bx, By, Bz, network)

(Tensor(shape=[], dtype=Float32, value= 0.684924),
 Tensor(shape=[], dtype=Float32, value= 0.413587),
 Tensor(shape=[], dtype=Float32, value= 0.267594),
 Tensor(shape=[], dtype=Float32, value= 0.409553),
 Tensor(shape=[], dtype=Float32, value= 0.275456),
 Tensor(shape=[], dtype=Float32, value= 2.05111))

In [11]:
def forward_fn(x,y,z, Bx,By,Bz):
    loss_b, loss_div, loss_cul, loss_BC_div, loss_BC_cul, loss  = loss_fn(x,y,z,Bx,By,Bz,network)
    return loss_b, loss_div, loss_cul, loss_BC_div, loss_BC_cul, loss

In [12]:
grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

In [13]:
def train_step(x,y,z, Bx,By,Bz):
    (loss_b, loss_div, loss_cul, loss_BC_div, loss_BC_cul, loss), grads = grad_fn(x,y,z,Bx,By,Bz)
    optimizer(grads)
    return loss_b, loss

In [16]:
def train_loop(model, x,y,z, Bx,By,Bz):
    model.set_train()
    loss_b, loss = train_step(x,y,z, Bx,By,Bz)
    print(loss_b, loss.asnumpy())

In [17]:
for t in range(10000):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(network, x,y,z, Bx,By,Bz)

Epoch 1
-------------------------------
0.56222713 1.7230891
Epoch 2
-------------------------------
0.45149943 1.4228619
Epoch 3
-------------------------------
0.36061528 1.1791465
Epoch 4
-------------------------------
0.30164677 0.9957106
Epoch 5
-------------------------------
0.28143126 0.88449776
Epoch 6
-------------------------------
0.28384688 0.8566459
Epoch 7
-------------------------------
0.28382328 0.8527546
Epoch 8
-------------------------------
0.26992014 0.8561909
Epoch 9
-------------------------------
0.24152063 0.8613559
Epoch 10
-------------------------------
0.20214522 0.8576337
Epoch 11
-------------------------------
0.15743136 0.84770024
Epoch 12
-------------------------------
0.116856314 0.8442113
Epoch 13
-------------------------------
0.096368015 0.84804976
Epoch 14
-------------------------------
0.100366466 0.8511165
Epoch 15
-------------------------------
0.10739727 0.8233143
Epoch 16
-------------------------------
0.10360655 0.7508311
Epoch 17
--